In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
#from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
test_demographics = pd.read_csv('data/testdemographics.csv')
test_performance = pd.read_csv('data/testperf.csv')
test_previousloans = pd.read_csv('data/testprevloans.csv')
train_demographics = pd.read_csv('data/traindemographics.csv')
train_performance = pd.read_csv('data/trainperf.csv')
train_previousloans = pd.read_csv('data/testprevloans.csv')

In [3]:
train_demographics['bank_account_type'].value_counts()

Savings    3425
Other       865
Current      56
Name: bank_account_type, dtype: int64

### Merging the datasets

In [4]:
train= pd.merge(train_performance, train_previousloans["firstduedate"], left_index= True, right_index= True)
train= pd.merge(train, train_previousloans["closeddate"], left_index= True, right_index= True)
train= pd.merge(train, train_previousloans["firstrepaiddate"], left_index= True, right_index= True)
train= pd.merge(train, train_demographics["birthdate"], left_index= True, right_index= True)
train= pd.merge(train, train_demographics["longitude_gps"], left_index= True, right_index= True)
train= pd.merge(train, train_demographics["latitude_gps"], left_index= True, right_index= True)
train

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag,firstduedate,closeddate,firstrepaiddate,birthdate,longitude_gps,latitude_gps
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,NaN,Good,2016-06-16 00:00:00.000000,2016-06-17 00:04:15.000000,2016-06-16 15:44:08.000000,1973-10-10 00:00:00.000000,3.319219,6.528604
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41.000000,2017-07-05 16:04:18.000000,15000.0,17250.0,30,NaN,Good,2017-03-06 00:00:00.000000,2017-03-02 16:22:58.000000,2017-03-02 16:07:47.000000,1986-01-21 00:00:00.000000,3.325598,7.119403
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57.000000,2017-07-06 13:52:51.000000,20000.0,22250.0,15,NaN,Good,2017-04-03 00:00:00.000000,2017-04-02 00:44:24.000000,2017-04-01 21:29:46.000000,1987-04-01 00:00:00.000000,5.746100,5.563174
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41.000000,2017-07-27 18:00:35.000000,10000.0,11500.0,15,NaN,Good,2016-07-04 00:00:00.000000,2016-07-04 11:34:04.000000,2016-07-04 11:19:01.000000,1991-07-19 00:00:00.000000,3.362850,6.642485
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45.000000,2017-07-03 22:42:39.000000,40000.0,44000.0,30,NaN,Good,2016-05-31 00:00:00.000000,2016-06-02 00:02:58.000000,2016-06-01 15:22:34.000000,1982-11-22 00:00:00.000000,8.455332,11.971410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,8a858f6c570f2d6601572a4acffc221f,302000403,7,2017-07-28 12:00:25.000000,2017-07-28 11:00:18.000000,20000.0,24500.0,30,NaN,Good,2017-06-29 00:00:00.000000,2017-06-28 13:22:52.000000,2017-06-28 13:12:43.000000,1985-12-13 00:00:00.000000,3.236753,7.030168
4342,8a858f735aec46c4015af026256357b8,301999973,5,2017-07-28 07:58:26.000000,2017-07-28 06:57:16.000000,20000.0,24500.0,30,NaN,Good,2017-06-08 00:00:00.000000,2017-06-06 10:26:05.000000,2017-06-06 10:15:54.000000,1982-07-01 00:00:00.000000,7.013749,4.875662
4343,8a858f7c5be8dc45015bea117aa57efe,301979154,3,2017-07-15 13:35:01.000000,2017-07-15 12:34:53.000000,10000.0,11500.0,15,NaN,Good,2017-04-17 00:00:00.000000,2017-04-06 16:30:10.000000,2017-04-06 16:15:00.000000,1989-09-26 00:00:00.000000,6.295530,7.092508
4344,8a858f8658c206a90158d0c447863597,301988349,8,2017-07-20 21:20:00.000000,2017-07-20 20:19:53.000000,30000.0,34500.0,30,NaN,Good,2017-05-30 00:00:00.000000,2017-06-02 09:17:10.000000,2017-06-02 09:07:02.000000,1985-09-06 00:00:00.000000,3.354206,6.539070


In [5]:
test_performance.drop(['approveddate','creationdate'],axis=1,inplace=True)

In [6]:
test= pd.merge(test_performance, test_previousloans["firstduedate"], left_index= True, right_index= True)
test= pd.merge(test, test_previousloans["closeddate"], left_index= True, right_index= True)
test= pd.merge(test, test_previousloans["firstrepaiddate"], left_index= True, right_index= True)
test= pd.merge(test, test_demographics["birthdate"], left_index= True, right_index= True)
test= pd.merge(test, test_demographics["longitude_gps"], left_index= True, right_index= True)
test= pd.merge(test, test_demographics["latitude_gps"], left_index= True, right_index= True)
test= pd.merge(test, test_previousloans["creationdate"], left_index= True, right_index= True)
test= pd.merge(test, test_previousloans["approveddate"], left_index= True, right_index= True)
test

,customerid,systemloanid,loannumber,loanamount,totaldue,termdays,referredby,firstduedate,closeddate,firstrepaiddate,birthdate,longitude_gps,latitude_gps,creationdate,approveddate
0,8a858899538ddb8e015390510b321f08,301998974,4,10000,12250.0,30,NaN,2016-06-16 00:00:00.000000,2016-06-17 00:04:15.000000,2016-06-16 15:44:08.000000,1976-08-28 00:00:00.000000,5.296628,7.593965,2016-05-17 09:36:55.000000,2016-05-17 10:37:00.000000
1,8a858959537a097401537a4e316e25f7,301963615,10,40000,44000.0,30,NaN,2017-03-06 00:00:00.000000,2017-03-02 16:22:58.000000,2017-03-02 16:07:47.000000,1978-06-23 00:00:00.000000,3.294513,6.596602,2017-02-04 20:28:52.000000,2017-02-04 21:28:59.000000
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,20000,24500.0,30,NaN,2017-04-03 00:00:00.000000,2017-04-02 00:44:24.000000,2017-04-01 21:29:46.000000,1984-04-04 00:00:00.000000,8.501912,7.729364,2017-03-04 09:28:16.000000,2017-03-04 10:28:22.000000
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,30000,34500.0,30,NaN,2016-07-04 00:00:00.000000,2016-07-04 11:34:04.000000,2016-07-04 11:19:01.000000,1983-05-28 00:00:00.000000,3.318904,6.681595,2016-06-02 13:27:08.000000,2016-06-02 14:27:14.000000
4,8a858e225a28c713015a30db5c48383d,301959177,4,20000,24500.0,30,NaN,2016-05-31 00:00:00.000000,2016-06-02 00:02:58.000000,2016-06-01 15:22:34.000000,1982-03-29 00:00:00.000000,6.354624,4.949031,2016-05-16 08:13:04.000000,2016-05-16 09:13:12.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,8a858fb45bb59c21015bb88a191f58f2,301991141,3,10000,11500.0,15,NaN,2017-04-24 00:00:00.000000,2017-05-24 15:49:31.000000,2017-04-25 10:28:06.000000,1980-08-05 00:00:00.000000,3.583009,6.493201,2017-03-24 17:27:29.000000,2017-03-24 18:27:35.000000
1446,8a858fcb5b00cc54015b0253ced26a5f,301963018,3,10000,13000.0,30,NaN,2017-07-10 00:00:00.000000,2017-07-08 05:10:32.000000,2017-07-08 05:00:21.000000,1994-02-25 00:00:00.000000,3.838206,7.352298,2017-06-08 16:02:26.000000,2017-06-08 17:03:35.000000
1447,8a858fde56eb02280156f59b976d46c0,301994653,9,40000,48000.0,60,NaN,2017-04-19 00:00:00.000000,2017-04-05 22:22:44.000000,2017-04-05 22:07:36.000000,1978-07-30 00:00:00.000000,3.448487,6.435382,2017-04-03 18:48:30.000000,2017-04-03 19:48:38.000000
1448,8a858e10570f2d65015717fcfec44996,301961483,14,60000,68100.0,90,NaN,2017-07-07 00:00:00.000000,2017-07-03 15:33:59.000000,2017-07-03 15:23:50.000000,1992-04-02 00:00:00.000000,7.036701,4.781278,2017-06-07 12:11:10.000000,2017-06-07 13:12:17.000000


In [7]:
train.isnull().values.any()

True

In [8]:
train.isnull().sum(axis = 0)

customerid            0
systemloanid          0
loannumber            0
approveddate          0
creationdate          0
loanamount            0
totaldue              0
termdays              0
referredby         3762
good_bad_flag         0
firstduedate          0
closeddate            0
firstrepaiddate       0
birthdate             0
longitude_gps         0
latitude_gps          0
dtype: int64

In [9]:
train = train.drop(['referredby'], axis=1)

In [10]:
train.columns

Index(['customerid', 'systemloanid', 'loannumber', 'approveddate',
       'creationdate', 'loanamount', 'totaldue', 'termdays', 'good_bad_flag',
       'firstduedate', 'closeddate', 'firstrepaiddate', 'birthdate',
       'longitude_gps', 'latitude_gps'],
      dtype='object')

In [11]:
train.dtypes

customerid          object
systemloanid         int64
loannumber           int64
approveddate        object
creationdate        object
loanamount         float64
totaldue           float64
termdays             int64
good_bad_flag       object
firstduedate        object
closeddate          object
firstrepaiddate     object
birthdate           object
longitude_gps      float64
latitude_gps       float64
dtype: object

In [12]:
train["birthdate"]= pd.to_datetime(train["birthdate"])
train["approveddate"]= pd.to_datetime(train["approveddate"])
train["creationdate"]= pd.to_datetime(train["creationdate"])
train["firstduedate"]= pd.to_datetime(train["firstduedate"])
train["firstrepaiddate"]= pd.to_datetime(train["firstrepaiddate"])
train["closeddate"]= pd.to_datetime(train["firstrepaiddate"])

In [13]:
test["birthdate"]= pd.to_datetime(test["birthdate"])
test["approveddate"]= pd.to_datetime(test["approveddate"])
test["creationdate"]= pd.to_datetime(test["creationdate"])
test["firstduedate"]= pd.to_datetime(test["firstduedate"])
test["firstrepaiddate"]= pd.to_datetime(test["firstrepaiddate"])
test["closeddate"]= pd.to_datetime(test["closeddate"])

### Percentage of good and bad

In [14]:
train.good_bad_flag.value_counts(normalize=True)

Good    0.783019
Bad     0.216981
Name: good_bad_flag, dtype: float64

In [15]:
train

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,good_bad_flag,firstduedate,closeddate,firstrepaiddate,birthdate,longitude_gps,latitude_gps
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56,2017-07-25 07:22:47,30000.0,34500.0,30,Good,2016-06-16,2016-06-16 15:44:08,2016-06-16 15:44:08,1973-10-10,3.319219,6.528604
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41,2017-07-05 16:04:18,15000.0,17250.0,30,Good,2017-03-06,2017-03-02 16:07:47,2017-03-02 16:07:47,1986-01-21,3.325598,7.119403
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57,2017-07-06 13:52:51,20000.0,22250.0,15,Good,2017-04-03,2017-04-01 21:29:46,2017-04-01 21:29:46,1987-04-01,5.746100,5.563174
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41,2017-07-27 18:00:35,10000.0,11500.0,15,Good,2016-07-04,2016-07-04 11:19:01,2016-07-04 11:19:01,1991-07-19,3.362850,6.642485
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45,2017-07-03 22:42:39,40000.0,44000.0,30,Good,2016-05-31,2016-06-01 15:22:34,2016-06-01 15:22:34,1982-11-22,8.455332,11.971410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,8a858f6c570f2d6601572a4acffc221f,302000403,7,2017-07-28 12:00:25,2017-07-28 11:00:18,20000.0,24500.0,30,Good,2017-06-29,2017-06-28 13:12:43,2017-06-28 13:12:43,1985-12-13,3.236753,7.030168
4342,8a858f735aec46c4015af026256357b8,301999973,5,2017-07-28 07:58:26,2017-07-28 06:57:16,20000.0,24500.0,30,Good,2017-06-08,2017-06-06 10:15:54,2017-06-06 10:15:54,1982-07-01,7.013749,4.875662
4343,8a858f7c5be8dc45015bea117aa57efe,301979154,3,2017-07-15 13:35:01,2017-07-15 12:34:53,10000.0,11500.0,15,Good,2017-04-17,2017-04-06 16:15:00,2017-04-06 16:15:00,1989-09-26,6.295530,7.092508
4344,8a858f8658c206a90158d0c447863597,301988349,8,2017-07-20 21:20:00,2017-07-20 20:19:53,30000.0,34500.0,30,Good,2017-05-30,2017-06-02 09:07:02,2017-06-02 09:07:02,1985-09-06,3.354206,6.539070


In [16]:
train["birth"]= [datetime.timestamp(i) for i in train.birthdate]
train["approved"]= [datetime.timestamp(i) for i in train.approveddate]
train["creation"]= [datetime.timestamp(i) for i in train.creationdate]
train["duedate"]= [datetime.timestamp(i) for i in train.firstduedate]
train["repaiddate"]= [datetime.timestamp(i) for i in train.firstrepaiddate]
train["closed"]= [datetime.timestamp(i) for i in train.closeddate]
train

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,good_bad_flag,firstduedate,...,firstrepaiddate,birthdate,longitude_gps,latitude_gps,birth,approved,creation,duedate,repaiddate,closed
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56,2017-07-25 07:22:47,30000.0,34500.0,30,Good,2016-06-16,...,2016-06-16 15:44:08,1973-10-10,3.319219,6.528604,119048400.0,1.500960e+09,1.500957e+09,1.466024e+09,1.466081e+09,1.466081e+09
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41,2017-07-05 16:04:18,15000.0,17250.0,30,Good,2017-03-06,...,2017-03-02 16:07:47,1986-01-21,3.325598,7.119403,506638800.0,1.499263e+09,1.499260e+09,1.488748e+09,1.488460e+09,1.488460e+09
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57,2017-07-06 13:52:51,20000.0,22250.0,15,Good,2017-04-03,...,2017-04-01 21:29:46,1987-04-01,5.746100,5.563174,544222800.0,1.499342e+09,1.499338e+09,1.491167e+09,1.491071e+09,1.491071e+09
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41,2017-07-27 18:00:35,10000.0,11500.0,15,Good,2016-07-04,...,2016-07-04 11:19:01,1991-07-19,3.362850,6.642485,679870800.0,1.501171e+09,1.501168e+09,1.467580e+09,1.467620e+09,1.467620e+09
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45,2017-07-03 22:42:39,40000.0,44000.0,30,Good,2016-05-31,...,2016-06-01 15:22:34,1982-11-22,8.455332,11.971410,406760400.0,1.499115e+09,1.499111e+09,1.464642e+09,1.464784e+09,1.464784e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,8a858f6c570f2d6601572a4acffc221f,302000403,7,2017-07-28 12:00:25,2017-07-28 11:00:18,20000.0,24500.0,30,Good,2017-06-29,...,2017-06-28 13:12:43,1985-12-13,3.236753,7.030168,503269200.0,1.501232e+09,1.501229e+09,1.498684e+09,1.498645e+09,1.498645e+09
4342,8a858f735aec46c4015af026256357b8,301999973,5,2017-07-28 07:58:26,2017-07-28 06:57:16,20000.0,24500.0,30,Good,2017-06-08,...,2017-06-06 10:15:54,1982-07-01,7.013749,4.875662,394318800.0,1.501218e+09,1.501214e+09,1.496869e+09,1.496733e+09,1.496733e+09
4343,8a858f7c5be8dc45015bea117aa57efe,301979154,3,2017-07-15 13:35:01,2017-07-15 12:34:53,10000.0,11500.0,15,Good,2017-04-17,...,2017-04-06 16:15:00,1989-09-26,6.295530,7.092508,622760400.0,1.500115e+09,1.500111e+09,1.492376e+09,1.491484e+09,1.491484e+09
4344,8a858f8658c206a90158d0c447863597,301988349,8,2017-07-20 21:20:00,2017-07-20 20:19:53,30000.0,34500.0,30,Good,2017-05-30,...,2017-06-02 09:07:02,1985-09-06,3.354206,6.539070,494802000.0,1.500575e+09,1.500571e+09,1.496092e+09,1.496384e+09,1.496384e+09


In [17]:
test["birth"]= [datetime.timestamp(i) for i in test.birthdate]
test["duedate"] = [datetime.timestamp(i) for i in test.firstduedate]
test["repaiddate"]= [datetime.timestamp(i) for i in test.firstrepaiddate]
test["approved"]= [datetime.timestamp(i) for i in test.approveddate]
test["creation"]= [datetime.timestamp(i) for i in test.creationdate]
test["closed"]= [datetime.timestamp(i) for i in test.closeddate]
test

,customerid,systemloanid,loannumber,loanamount,totaldue,termdays,referredby,firstduedate,closeddate,firstrepaiddate,...,longitude_gps,latitude_gps,creationdate,approveddate,birth,duedate,repaiddate,approved,creation,closed
0,8a858899538ddb8e015390510b321f08,301998974,4,10000,12250.0,30,NaN,2016-06-16,2016-06-17 00:04:15,2016-06-16 15:44:08,...,5.296628,7.593965,2016-05-17 09:36:55,2016-05-17 10:37:00,210027600.0,1.466024e+09,1.466081e+09,1.463471e+09,1.463467e+09,1.466111e+09
1,8a858959537a097401537a4e316e25f7,301963615,10,40000,44000.0,30,NaN,2017-03-06,2017-03-02 16:22:58,2017-03-02 16:07:47,...,3.294513,6.596602,2017-02-04 20:28:52,2017-02-04 21:28:59,267397200.0,1.488748e+09,1.488460e+09,1.486233e+09,1.486229e+09,1.488461e+09
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,20000,24500.0,30,NaN,2017-04-03,2017-04-02 00:44:24,2017-04-01 21:29:46,...,8.501912,7.729364,2017-03-04 09:28:16,2017-03-04 10:28:22,449874000.0,1.491167e+09,1.491071e+09,1.488613e+09,1.488609e+09,1.491083e+09
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,30000,34500.0,30,NaN,2016-07-04,2016-07-04 11:34:04,2016-07-04 11:19:01,...,3.318904,6.681595,2016-06-02 13:27:08,2016-06-02 14:27:14,422917200.0,1.467580e+09,1.467620e+09,1.464867e+09,1.464863e+09,1.467621e+09
4,8a858e225a28c713015a30db5c48383d,301959177,4,20000,24500.0,30,NaN,2016-05-31,2016-06-02 00:02:58,2016-06-01 15:22:34,...,6.354624,4.949031,2016-05-16 08:13:04,2016-05-16 09:13:12,386197200.0,1.464642e+09,1.464784e+09,1.463379e+09,1.463376e+09,1.464815e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,8a858fb45bb59c21015bb88a191f58f2,301991141,3,10000,11500.0,15,NaN,2017-04-24,2017-05-24 15:49:31,2017-04-25 10:28:06,...,3.583009,6.493201,2017-03-24 17:27:29,2017-03-24 18:27:35,334270800.0,1.492981e+09,1.493105e+09,1.490369e+09,1.490366e+09,1.495630e+09
1446,8a858fcb5b00cc54015b0253ced26a5f,301963018,3,10000,13000.0,30,NaN,2017-07-10,2017-07-08 05:10:32,2017-07-08 05:00:21,...,3.838206,7.352298,2017-06-08 16:02:26,2017-06-08 17:03:35,762123600.0,1.499634e+09,1.499479e+09,1.496931e+09,1.496927e+09,1.499480e+09
1447,8a858fde56eb02280156f59b976d46c0,301994653,9,40000,48000.0,60,NaN,2017-04-19,2017-04-05 22:22:44,2017-04-05 22:07:36,...,3.448487,6.435382,2017-04-03 18:48:30,2017-04-03 19:48:38,270594000.0,1.492549e+09,1.491419e+09,1.491238e+09,1.491235e+09,1.491420e+09
1448,8a858e10570f2d65015717fcfec44996,301961483,14,60000,68100.0,90,NaN,2017-07-07,2017-07-03 15:33:59,2017-07-03 15:23:50,...,7.036701,4.781278,2017-06-07 12:11:10,2017-06-07 13:12:17,702162000.0,1.499375e+09,1.499085e+09,1.496830e+09,1.496827e+09,1.499085e+09


In [18]:
train= train.drop(["birthdate","creationdate","approveddate","firstduedate","firstrepaiddate","closeddate"], axis= 1)
test= test.drop(["birthdate","firstduedate","firstrepaiddate","closeddate"], axis= 1)

In [19]:
test

,customerid,systemloanid,loannumber,loanamount,totaldue,termdays,referredby,longitude_gps,latitude_gps,creationdate,approveddate,birth,duedate,repaiddate,approved,creation,closed
0,8a858899538ddb8e015390510b321f08,301998974,4,10000,12250.0,30,NaN,5.296628,7.593965,2016-05-17 09:36:55,2016-05-17 10:37:00,210027600.0,1.466024e+09,1.466081e+09,1.463471e+09,1.463467e+09,1.466111e+09
1,8a858959537a097401537a4e316e25f7,301963615,10,40000,44000.0,30,NaN,3.294513,6.596602,2017-02-04 20:28:52,2017-02-04 21:28:59,267397200.0,1.488748e+09,1.488460e+09,1.486233e+09,1.486229e+09,1.488461e+09
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,20000,24500.0,30,NaN,8.501912,7.729364,2017-03-04 09:28:16,2017-03-04 10:28:22,449874000.0,1.491167e+09,1.491071e+09,1.488613e+09,1.488609e+09,1.491083e+09
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,30000,34500.0,30,NaN,3.318904,6.681595,2016-06-02 13:27:08,2016-06-02 14:27:14,422917200.0,1.467580e+09,1.467620e+09,1.464867e+09,1.464863e+09,1.467621e+09
4,8a858e225a28c713015a30db5c48383d,301959177,4,20000,24500.0,30,NaN,6.354624,4.949031,2016-05-16 08:13:04,2016-05-16 09:13:12,386197200.0,1.464642e+09,1.464784e+09,1.463379e+09,1.463376e+09,1.464815e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,8a858fb45bb59c21015bb88a191f58f2,301991141,3,10000,11500.0,15,NaN,3.583009,6.493201,2017-03-24 17:27:29,2017-03-24 18:27:35,334270800.0,1.492981e+09,1.493105e+09,1.490369e+09,1.490366e+09,1.495630e+09
1446,8a858fcb5b00cc54015b0253ced26a5f,301963018,3,10000,13000.0,30,NaN,3.838206,7.352298,2017-06-08 16:02:26,2017-06-08 17:03:35,762123600.0,1.499634e+09,1.499479e+09,1.496931e+09,1.496927e+09,1.499480e+09
1447,8a858fde56eb02280156f59b976d46c0,301994653,9,40000,48000.0,60,NaN,3.448487,6.435382,2017-04-03 18:48:30,2017-04-03 19:48:38,270594000.0,1.492549e+09,1.491419e+09,1.491238e+09,1.491235e+09,1.491420e+09
1448,8a858e10570f2d65015717fcfec44996,301961483,14,60000,68100.0,90,NaN,7.036701,4.781278,2017-06-07 12:11:10,2017-06-07 13:12:17,702162000.0,1.499375e+09,1.499085e+09,1.496830e+09,1.496827e+09,1.499085e+09


In [20]:
train

,customerid,systemloanid,loannumber,loanamount,totaldue,termdays,good_bad_flag,longitude_gps,latitude_gps,birth,approved,creation,duedate,repaiddate,closed
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,30000.0,34500.0,30,Good,3.319219,6.528604,119048400.0,1.500960e+09,1.500957e+09,1.466024e+09,1.466081e+09,1.466081e+09
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,15000.0,17250.0,30,Good,3.325598,7.119403,506638800.0,1.499263e+09,1.499260e+09,1.488748e+09,1.488460e+09,1.488460e+09
2,8a8588f35438fe12015444567666018e,301966580,7,20000.0,22250.0,15,Good,5.746100,5.563174,544222800.0,1.499342e+09,1.499338e+09,1.491167e+09,1.491071e+09,1.491071e+09
3,8a85890754145ace015429211b513e16,301999343,3,10000.0,11500.0,15,Good,3.362850,6.642485,679870800.0,1.501171e+09,1.501168e+09,1.467580e+09,1.467620e+09,1.467620e+09
4,8a858970548359cc0154883481981866,301962360,9,40000.0,44000.0,30,Good,8.455332,11.971410,406760400.0,1.499115e+09,1.499111e+09,1.464642e+09,1.464784e+09,1.464784e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,8a858f6c570f2d6601572a4acffc221f,302000403,7,20000.0,24500.0,30,Good,3.236753,7.030168,503269200.0,1.501232e+09,1.501229e+09,1.498684e+09,1.498645e+09,1.498645e+09
4342,8a858f735aec46c4015af026256357b8,301999973,5,20000.0,24500.0,30,Good,7.013749,4.875662,394318800.0,1.501218e+09,1.501214e+09,1.496869e+09,1.496733e+09,1.496733e+09
4343,8a858f7c5be8dc45015bea117aa57efe,301979154,3,10000.0,11500.0,15,Good,6.295530,7.092508,622760400.0,1.500115e+09,1.500111e+09,1.492376e+09,1.491484e+09,1.491484e+09
4344,8a858f8658c206a90158d0c447863597,301988349,8,30000.0,34500.0,30,Good,3.354206,6.539070,494802000.0,1.500575e+09,1.500571e+09,1.496092e+09,1.496384e+09,1.496384e+09


In [21]:
train.drop('customerid',axis=1,inplace=True)

In [22]:
train['good_bad_flag']

0       Good
1       Good
2       Good
3       Good
4       Good
        ... 
4341    Good
4342    Good
4343    Good
4344    Good
4345    Good
Name: good_bad_flag, Length: 4346, dtype: object

In [23]:
train.dtypes

systemloanid       int64
loannumber         int64
loanamount       float64
totaldue         float64
termdays           int64
good_bad_flag     object
longitude_gps    float64
latitude_gps     float64
birth            float64
approved         float64
creation         float64
duedate          float64
repaiddate       float64
closed           float64
dtype: object

### Modelling

In [24]:
le= LabelEncoder()
train["good_bad_flag"]= le.fit_transform(train["good_bad_flag"].values)

In [25]:
X = train.drop('good_bad_flag', axis=1)
Y = train.good_bad_flag

In [26]:
train_x, val_x, train_y, val_y = train_test_split(X,Y, test_size=0.2, random_state = 42)

In [27]:
randomforest = RandomForestClassifier(n_estimators=300, random_state=0)
adaboost = AdaBoostClassifier(n_estimators=200, learning_rate=0.1)
randomforest.fit(train_x,train_y)
prediction=randomforest.predict(val_x)
print(f"Accuracy: {accuracy_score(prediction,val_y)}")

Accuracy: 0.7586206896551724


In [28]:
adaboost.fit(train_x, train_y)
pred= adaboost.predict(val_x)
print(1- (accuracy_score(pred, val_y)))

0.24022988505747123


In [29]:
customerid = test.pop('customerid')

In [30]:
test.drop(['creationdate','approveddate','referredby'],axis=1,inplace=True)

In [31]:
test_y = adaboost.predict(test)

In [32]:
results = pd.DataFrame({'customerid':customerid,
                        'Good_Bad_flag': test_y})


In [33]:
results.to_csv('results.csv', index= False)